In [3]:
import pandas as pd
import numpy as np
import os
import sys


from datetime import datetime
from src.utils import choose_device
# Make `src` dir can be imported
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # /aicup-predict-energy-generation
sys.path.append(project_root_path)  

from src.fe_tools import create_time_features


In [38]:
df_raw = pd.read_csv('../data/processed_data/combined_data.csv')
df_raw = create_time_features(df_raw, "datetime")
df_raw['date'] = pd.to_datetime(df_raw['date'])

df_device = choose_device(df_raw, 'L10')

In [39]:
df_device.head()

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9


In [56]:
def create_lag_fe(df: pd.DataFrame, col, group_col, lag_step):
    if df.columns.__contains__(col):
        df[f'lag_{col}'] = df.groupby([group_col])[col].shift(lag_step).fillna(df[col])
        return df
    else:
        raise ValueError(f"{col} doesn't exsit.")
    

In [58]:
def feature_interaction(df, col1, col2):

    df[f'multi_{col1}_{col2}'] = df[col1] * df[col2]

    return df

In [57]:
create_lag_fe(df_device, 'sunlight', 'date', 1)

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,lag_sunlight
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,652.92
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,652.92
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,682.50
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,750.00
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,738.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30,90.83
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30,102.50
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30,115.00
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30,129.17


In [59]:
feature_interaction(df_device, 'temperature', 'humidity')

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,lag_sunlight,multi_temperature_humidity
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,652.92,1470.1370
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,652.92,1472.6664
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,682.50,1481.1340
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,750.00,1484.7402
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,738.33,1486.4640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30,90.83,2427.3792
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30,102.50,2429.9086
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30,115.00,2432.6456
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30,129.17,2433.5344
